# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes. As usual, we run some import statements:

In [1]:
!pip install --upgrade systemml

  Using cached https://files.pythonhosted.org/packages/ec/d4/146cd03f62945bcb1f7cf8fca743e1eeca765e79a0cffe35ce37a65cd78a/systemml-1.1.0.tar.gz
    100% |████████████████████████████████| 13.8MB 93kB/s  eta 0:00:01   44% |██████████████▎                 | 6.2MB 57.1MB/s eta 0:00:01
    100% |████████████████████████████████| 33.1MB 38kB/s  eta 0:00:01
    100% |████████████████████████████████| 8.7MB 152kB/s eta 0:00:01
    100% |████████████████████████████████| 4.9MB 228kB/s eta 0:00:01    30% |█████████▉                      | 1.5MB 51.9MB/s eta 0:00:01
    100% |████████████████████████████████| 2.0MB 636kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/30/4e/27c34b62430286c6d59177a0842ed90dc789ce5d1ed740887653b898779a/pytz-2018.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cf/f5/af2b09c957ace60dcfac112b669c45c8c97e32f94aa8b56da4c6d1682825/python_dateutil-2.7.3-py2.py3-none-any.whl
Requirement already up-to-date: six>=1.5 in /us

In [2]:
from systemml import MLContext, dml
import numpy as np
import time

Then we create an MLContext to interface with Apache SystemML. Note that we pass a SparkSession object as parameter so SystemML now knows how to talk to the Apache Spark cluster

In [3]:
ml = MLContext(spark)

Now we create some large random matrices to have numpy and SystemML crunch on it

In [4]:
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are not familiar with matrix-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [6]:
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print (time.time()-start)

0.46599507331848145


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [7]:
u = np.random.rand(10000,100000)
s = np.random.rand(100000,10000)
w = np.random.rand(10000,10000)

After a short while you should see a memory error. This is because the operating system process was not able to allocate enough memory for storing the numpy array on the heap. Now it's time to re-implement the very same operations as DML in SystemML, and this is your task. Just replace all ###your_code_goes_here sections with proper code, please consider the following table which contains all DML syntax you need:

| Syntax        |            |   
| ------------- |-------------| 
| t(M)      | transpose of a matrix, where M is the matrix | 
| %*% | matrix-matrix multiplication      |  
| * | scalar multiplication      |   

## Task

In [12]:
script = """
res = sum(t(U) %*% (W * (U %*% S)))
"""

To get consistent results we switch from a random matrix initialization to something deterministic

In [13]:
u = np.arange(100000).reshape((100, 1000))
s = np.arange(100000).reshape((1000, 100))
w = np.arange(10000).reshape((100, 100))

In [14]:
prog = dml(script).input('U', u).input('S', s).input('W', w).output('res')
res = ml.execute(prog).get('res')
print (res)

SystemML Statistics:
Total execution time:		0.061 sec.
Number of executed Spark inst:	3.


1.2526052592176266e+28


If everything runs fine you should get *1.25260525922e+28* as result. Feel free to submit your DML script to the grader now!

### Submission

In [15]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-07-26 01:44:25--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2029 (2.0K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,029       --.-K/s   in 0s      

2018-07-26 01:44:25 (17.1 MB/s) - ‘rklib.py’ saved [2029/2029]



In [16]:
from rklib import submit
key = "esRk7vn-Eeej-BLTuYzd0g"
part = "fUxc8"

email = "priyankaraina.sg@gmail.com"
secret = "vVceflqRrusfnjm4"
submit(email, secret, key, part, [part], script)

Submission successful, please check on the coursera grader page for the status
